In [1]:
import scipy.io as sio
from sklearn import preprocessing
import parmap
import numpy as np

lb = preprocessing.LabelBinarizer()
lb.fit([1,2,3,4,5,6,7,8,9,10])
train = sio.loadmat("train.mat")
test = sio.loadmat("test.mat")
label = train['y']
img = train['X']
label = np.ravel(label)
# Process the label so that it can be read by tensorflow
# label = lb.transform(label)

Xtest = test['X']
ytest = test['y']
ytest = np.ravel(ytest)
# ytest = lb.transform(ytest)

In [2]:
# Processing includes converting to gray scale
# and binirize image
from skimage.filters import threshold_adaptive
import numpy as np

def process_(i, img):
    currentImg = img[:,:,:,i]

    # Grayscale image
    grayscale = np.dot(currentImg[:,:,:3], [0.299, 0.587, 0.114])

    # Binirization
    block_size = 15

    binary_adaptive = threshold_adaptive(grayscale, block_size, offset=10)

    placeholder = np.ravel(binary_adaptive)
    return placeholder
def process_hog(i,img):
    
    currentImg = img[:,:,:,i]

    # Grayscale image
    grayscale = np.dot(currentImg[:,:,:3], [0.299, 0.587, 0.114])

    # Binirization
    block_size = 15
    binary_adaptive = threshold_adaptive(grayscale, block_size, offset=10)
    
    count0 = np.where 

    # Extract HOG feature
    winSize = (16,16)
    blockSize = (16,16)
    blockStride = (8,8)
    cellSize = (4,4)
    nbins = 9
    derivAperture = 1
    winSigma = 4.
    histogramNormType = 0
    L2HysThreshold = 2.0000000000000001e-01
    gammaCorrection = 0
    nlevels = 64
    import cv2
    hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,
                            histogramNormType,L2HysThreshold,gammaCorrection,nlevels)
    hist = hog.compute(binary_adaptive.astype(np.uint8))
    return np.ravel(hist)

In [3]:
currentImg = img[:,:,:,i]

# Grayscale image
grayscale = np.dot(currentImg[:,:,:3], [0.299, 0.587, 0.114])

# Binirization
block_size = 15
binary_adaptive = threshold_adaptive(grayscale, block_size, offset=10)

NameError: name 'i' is not defined

In [4]:
# %timeit process(img)
import multiprocessing as mp
# temp = np.zeros((32*32,img.shape[3]), dtype=float)
num_img = img.shape[3]
num_img_test = Xtest.shape[3]
processed = parmap.map(process_hog, range(0,num_img), img)
processed_test = parmap.map(process_hog, range(0, num_img_test), Xtest)

In [79]:
print processed[2].shape

(3600,)


In [5]:
s = np.random.RandomState(43)

processed = np.array(processed)
processed_test = np.array(processed_test)

indxtrain = np.arange(num_img)
s.shuffle(indxtrain)

idxtest = np.arange(num_img_test)
s.shuffle(idxtest)


In [6]:
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
import sklearn.tree as tr


N=num_img_test
# clf = SGDClassifier(loss="squared_hinge", n_jobs=3, n_iter=10)
clf = tr.DecisionTreeClassifier(criterion='entropy')


clf.fit(processed, label)
predicted = clf.predict(processed_test[idxtest[:N],:])

print("Classification report for classifier %s:\n%s\n"
      % (clf, metrics.classification_report(ytest[idxtest[:N]], predicted)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(ytest[idxtest[:N]], predicted))

Classification report for classifier DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'):
             precision    recall  f1-score   support

          1       0.67      0.63      0.65      5099
          2       0.63      0.61      0.62      4149
          3       0.42      0.44      0.43      2882
          4       0.42      0.42      0.42      2523
          5       0.49      0.52      0.51      2384
          6       0.42      0.42      0.42      1977
          7       0.52      0.52      0.52      2019
          8       0.34      0.35      0.35      1660
          9       0.39      0.41      0.40      1595
         10       0.46      0.47      0.47      1744

avg / total       0.51      0.51      0.51     26032


Confusion matrix:
[[3219  223  259  328  103 

In [51]:
# Non linear SVM

from sklearn import svm, metrics
K = 20000
N = 1000
classifier = svm.SVC(gamma=0.001)
trainImg = processed[indxtrain[:K],:]
print trainImg.shape
testImg = processed_test[idxtest[:N],:]
print testImg.shape
classifier.fit(trainImg, label[indxtrain[:K]])

predicted = classifier.predict(testImg)

print("Classification report for classifier %s:\n%s\n"
      % (classifier, metrics.classification_report(ytest[idxtest[:N]], predicted)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(ytest[idxtest[:N]], predicted))


(20000, 324)
(1000, 324)
Classification report for classifier SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False):
             precision    recall  f1-score   support

          1       0.55      0.85      0.67       183
          2       0.58      0.70      0.64       152
          3       0.44      0.59      0.50       113
          4       0.51      0.50      0.50       112
          5       0.53      0.60      0.56        87
          6       0.68      0.36      0.47        78
          7       0.84      0.56      0.68        87
          8       0.86      0.26      0.40        70
          9       0.71      0.29      0.41        58
         10       0.88      0.38      0.53        60

avg / total       0.62      0.57      0.56      1000


Confusion matrix:
[[155  12   6   5   1   2   1   0   0   1]
 [ 16 107   8